# Puget Sound Regional Council Household Travel Survey (PSRC HTS) Tutorial

PSRC is the metroplitan planning organization (MPO) that spans King, Kitsap, Pierce and Snohomish County in Washington State. Every few years, they conduct a regional household travel survey collecting both socioedmographic and travel behavior data of a small sample of individuals in the region.

Please obtain the PSRC HTS data from their website for the persons, household, and trips level. Additional information is available for further reading.
https://household-travel-survey-psregcncl.hub.arcgis.com/

Surveys are collected either via a smartphone app or online. Additional trace data is generated from the smartphone survey (only available 2017/2019). Online survey data generates rough O/D lat/lon information (available for 2017, 2019, and 2021).


## Step 1: Read in datasets

**Trips:** trip-level information, such as depart/arrival time, origin/dest purpose, O/D census tracts, length of trip, speed, mode, etc.

**Persons:** person-level information, such as age, race, gender, employment status, industry, etc. 

**Households:** household-level information, such as size, number of children, lifecycle, location of home, attiudinal characteristics, etc.

See the codebook (at above link) for all available variables and decoding.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from statsmodels.stats.weightstats import DescrStatsW
import geopandas as gpd
from shapely.geometry import Point
import requests
from pyproj import CRS
import geopy.distance
import math

In [ ]:
# load datasets
# download these fro PSRC's website
trips = pd.read_csv("data/Household_Travel_Survey_Trips.csv")
persons = pd.read_csv("data/Household_Travel_Survey_Persons.csv")
households = pd.read_csv("data/Household_Travel_Survey_Households.csv")

# download this from github-- PRIVATE
od = pd.read_csv("data/od_data/trips_2017_2019_2021_locations.csv")

In [ ]:
# what information do we have?
for col in persons.columns: print(col)

# Step 2: Clean data

#### What do we need to clean?
- Outliers (travel time, spatial)
- Research-question specific (weekend data, specific trip purposes, etc.)
- Missing/no repsonse 

In [ ]:
# let's check weekend data. codebook say 1 is Monday, 7 is Sunday.
trips.daynum.value_counts(normalize=True)

Looks like our weekend trips are underrepresented, only accounting for ~18% of the data (assuming people make equal trips each day, ~28.6% would be representative).

Let's see how travel time looks...

In [ ]:
sns.histplot(trips.travel_time)
plt.xlim(0, 200)
plt.xlabel("Travel Time (minutes)")
plt.title(f"Distribution of Travel Times (n={len(trips)})")
plt.axvline(trips.travel_time.mean(), color='red', linestyle='dashed', linewidth=1)

# note: this is UNWEIGHTED and should not be interpreted for region-wide analysis

In [ ]:
trips.travel_time.describe()

Looks like we have a pretty long tail of travel times. Also notice some respondent bias-- people tend to think of their travel time in intervals of five. 

Since a lot of accessibility/mobility metrics can be derived from travel time, let's check the quality of the data.

In [ ]:
# check for missing values
trips.travel_time.isna().value_counts() # uh oh, 63,000 missing values?????

In [ ]:
# are we missing as many datetime strings?
print(trips.arrival_time_string.isna().value_counts(), '\n') # looks a lot better, only 6 missing
print(trips.depart_time_string.isna().value_counts()) # only 20 missing


In [ ]:
# let's drop the missing datetime strings
trips_clean = trips.dropna(subset=['depart_time_string', 'arrival_time_string']) # this only drops 23 observations

In [ ]:
# let's manually calculate them using the start and end times. this way we can preserve many more observations.

def travel_time_manual(row):
    ''' 
    function to calculate travel time from datetime arrival/depart strings.
    input: row of dataframe
    output: travel time in minutes
    '''
    date_str_arr = row.arrival_time_string[:-1]
    date_str_dep = row.depart_time_string[:-1]
    date_format = "Date: %Y-%m-%d %H:%M:%S.%f"

    # convert string to datetime object
    dt_object_arr = datetime.strptime(date_str_arr, date_format)
    dt_object_dep = datetime.strptime(date_str_dep, date_format)
    
    # calculate travel time
    delta_time = abs(dt_object_arr - dt_object_dep)
    travel_time_mins = delta_time.total_seconds()/60

    return travel_time_mins

In [ ]:
trips_clean.loc[:, "travel_time_calc"] = trips_clean.apply(travel_time_manual, axis=1)

In [ ]:
trips_clean.travel_time_calc.isna().value_counts() # no missing values


To create a final, cleaned dataset, let's say we want to look at just workday trips. We can assume that a "normal" trip is under 2 hours (adjust this for your purposes) since people need to go to schoo/work.

In [ ]:
# filter dataset
trips_clean = trips_clean[(trips_clean.travel_time_calc < 120) & (trips_clean.daynum.isin([1,2,3,4,5]))]

In [ ]:
# how much is retained?
print(f"{len(trips_clean)/len(trips)*100:.2f}% of the original dataset is retained after filtering.")

# Step 3. Summary Statistics

Ok, so the above give us an idea of the *sample* behaviors, but how can we generalize this to the entire Puget Sound region?
- Survey weights!
    - Generally, apply weights to each year. See codebook for more detail.
    - Note: survey weights are used so that the sampled individuals match ACS data, and that trip modes/purposes are representative. Read more here: https://www.psrc.org/media/3631

Using survey weights, we can make accurate comparisons between years when describing our data. Weights are needed for things such as visualizations and summary statistics, but not for models (since in models we want to understand relationships, which does not require representativeness).



Here's a cheatsheet for weighting:
| Summary level | 2017 | 2019 | 2017/2019 | 2021
| --- | --- | --- | --- | --- |
| Household | hh_weight_2017_v2021 | hh_weight_2019_v2021 | hh_weight_2017_2019_v2021 | hh_weight_2021_ABS |
| Person | hh_weight_2017* | hh_weight_2019* | hh_weight_2017_2019* | person_adult_weight_2021 |
| Trip | trip_weight_2017_v2021* | trip_weight_2019_v2021* | trip_weight_2017_2019_v2021* | trip_weight_2021_ABS_Panel_adult |
| Respondent | | | | Apply to articular questions** |

Notice there are some adult weights too-- 2021 sample does not contain children, while 2017/2019 does. So to compare trips between 2017/2019 and 2021, need to filter to only adult trips before applying weights. DO NOT COMPARE TRIPS WEIGHTED WITH CHILDREN TO WEIGHTS THAT DO NOT CONSIDER CHILDREN.

*: filter to adults only first to compare to 2021

**: respondent weights only apply to 2021 sample for
- workplace_pre_covid
- commute_freq_pre_covid
- commute_mode_pre_covid
- telecommute_freq_pre_covid
- employment_change_employer
- employment_change_location
- employment_change_new_job
- employment_change_laid_off
- employment_change_left_workforce
- employment_change_none

In [ ]:
# notice that person, household, and trip dfs all have survey weights
trips_clean.iloc[:, -8:-2]

In [ ]:
# so, how do we apply weights?
# visually
sns.kdeplot(trips_clean[trips_clean.survey_year ==2017].travel_time_calc, label="Unweighted")
sns.kdeplot(trips_clean[trips_clean.survey_year ==2017].travel_time_calc, weights=trips_clean.trip_weight_2017_v2021, label="Weighted")
plt.xlim(-10, 120)
plt.legend()
plt.title("Distribution of Travel Times in 2017, Weight Comparison")

In [ ]:
# now, with weights, we can compare trips between years

# filter 2017/2019 to be adult only
trips_clean_adults = trips_clean[(trips_clean.age != "Under 5 years old") | (trips_clean.age != "5-11 years") | 
                                 (trips_clean.age != "12-15 years") | (trips_clean.age != "16-17 years")]

# now we can compare weighted trips across years
sns.kdeplot(trips_clean_adults[trips_clean_adults.survey_year ==2017].travel_time_calc, weights=trips_clean_adults.trip_weight_2017_v2021, label="2017")
sns.kdeplot(trips_clean_adults[trips_clean_adults.survey_year ==2019].travel_time_calc, weights=trips_clean_adults.trip_weight_2019_v2021, label="2019")
sns.kdeplot(trips_clean_adults[trips_clean_adults.survey_year ==2021].travel_time_calc, weights=trips_clean_adults.trip_weight_2021_ABS_Panel_adul, label="2021")
plt.legend()

In [ ]:
# weighted descriptive statistics
trips_clean_2017 = trips_clean[trips_clean.survey_year == 2017]

# from statsmodels
ws_tt = DescrStatsW(trips_clean_2017.travel_time_calc, weights=trips_clean_2017.trip_weight_2017_v2021)
quantiles = ws_tt.quantile(probs=np.array([0.25,0.5,0.75]), return_pandas=False)
desc_stats = {"mean": ws_tt.mean, "std": ws_tt.std, "25th percentile": quantiles[0],
              "50th percentile": quantiles[1], "75th percentile": quantiles[2],
              "min": trips_clean_2017.travel_time_calc.min(), "max": trips_clean_2017.travel_time_calc.max()}
desc_stats 

# Step 4: Mobility metrics
Let's calculate some metrics on the daily, person level
- number of trips
- VMT
- person miles traveled
- trip chains


In [ ]:
# need data for each person each day
person_day_df = trips_clean.groupby(['person_dim_id', 'daynum']).size().reset_index()[['person_dim_id', 'daynum']]
person_day_df

In [ ]:
def mobility_metrics(row):
    ''' 
    function that calculates mobility metrics
    input: df row
    output: pandas series of metrics
    '''
    # filter to obtain trips that correspond with each person
    trips_day_person = trips_clean[(trips_clean.person_dim_id == row.person_dim_id) & (trips_clean.daynum == row.daynum)]

    # number of trips
    num_trips = len(trips_day_person)

    # person miles traveled
    pmt = trips_day_person.trip_path_distance.sum()

    # vehicle miles traveled
    vmt = trips_day_person.loc[trips_day_person['mode_simple'] == 'Drive', 'trip_path_distance'].sum()

    # trip chains
    trip_chains = 0
    start_home = False
    for idx, row in trips_day_person.iterrows():
        if row.origin_purpose == "Went home":
            start_home = True
        if (row.dest_purpose != "Went home") and start_home: # person continues their trip chain
            continue
        elif (row.dest_purpose == "Went home") and (row.origin_purpose == "Went home"): # home-home trip, not counted as a trip chain
            continue
        elif (row.dest_purpose == "Went home") and start_home: # end of the trip chain
            trip_chains += 1
            start_home = False # reset flag
    
    return pd.Series([num_trips, pmt, vmt, trip_chains])


In [ ]:
person_day_df[['num_trips', 'pmt', 'vmt', 'trip_chains']] = person_day_df.apply(mobility_metrics, axis=1)
person_day_df

In [ ]:
# visualize VMT-- let's not use weights here since it gets a little tricky since
# we're at both the person and trip level
plt.hist(person_day_df.vmt)
plt.show()

# still looks like there's an outlier...let's drop it
person_day_df_clean = person_day_df[person_day_df.vmt < 500]
plt.hist(person_day_df_clean.vmt, bins=100)
plt.xlim(0, 100)
plt.xlabel("Vehicle Miles Traveled")
plt.ylabel("Count")
plt.show()

In [ ]:
# what about vmt by gender?
# merge person data with person_day_df
person_day_df_clean = person_day_df_clean.merge(persons[["person_id", "gender"]], left_on='person_dim_id', 
                                                right_on="person_id", how='left')

In [ ]:
# let's drop some outliers
person_day_df_clean = person_day_df_clean[person_day_df_clean.vmt < 100]

# let's split up by gender
person_day_df_clean_fem = person_day_df_clean[person_day_df_clean["gender"] == "Female"]
person_day_df_clean_nonfem = person_day_df_clean[person_day_df_clean["gender"] != "Female"]

sns.boxplot(data=[person_day_df_clean_fem.vmt, person_day_df_clean_nonfem.vmt])
plt.xticks(ticks=[0,1], labels=["Female", "Non-female"])

print("Female mean VMT:", person_day_df_clean_fem.vmt.mean(), "\nMale mean VMT:", person_day_df_clean_nonfem.vmt.mean())
# can perform t-test to determine significance

# Bonus: O/D data + mapping
We can also use O/D data collected from the online survey to calculate some more spatial metrics, such as radius of gyration (activity space).

However, we need to clean this data too!

In [ ]:
# filter out trips outside WA
def are_points_inside_washington(origin_lat, origin_lng, dest_lat, dest_lng):
    washington_bbox = {
    'lat_min': 45.5435,
    'lat_max': 49.0025,
    'lon_min': -124.848974,
    'lon_max': -116.916197
    }
    return (washington_bbox['lat_min'] <= origin_lat <= washington_bbox['lat_max']) and \
           (washington_bbox['lon_min'] <= origin_lng <= washington_bbox['lon_max']) and \
           (washington_bbox['lat_min'] <= dest_lat <= washington_bbox['lat_max']) and \
           (washington_bbox['lon_min'] <= dest_lng <= washington_bbox['lon_max'])

In [ ]:
# create gdf for O/D trips
gdf_od = gpd.GeoDataFrame(od, geometry=gpd.points_from_xy(od['origin_lng'], od['origin_lat']), crs={'init':'EPSG:4326'})

# get rid of nas
gdf_od = gdf_od.dropna(subset=['origin_lng', 'origin_lat', 'dest_lng', 'dest_lat'])

# limit data to washington state
gdf_od['Both_Points_Inside_Washington'] = gdf_od.apply(lambda row: are_points_inside_washington(row['origin_lat'],
                                                                                         row['origin_lng'],
                                                                                         row['dest_lat'],
                                                                                         row['dest_lng']), axis=1)
gdf_od = gdf_od[gdf_od.Both_Points_Inside_Washington == True]

In [ ]:
# load in washington state counties
counties_url = "https://gisdata.kingcounty.gov/arcgis/rest/services/OpenDataPortal/politicl___base/MapServer/122/query?outFields=*&where=1%3D1&f=geojson"
response = requests.get(counties_url)
data = response.json()
gdf_counties = gpd.GeoDataFrame.from_features(data['features'], crs={'init':'EPSG:4326'})

# adjust crs
gdf_counties = gdf_counties.to_crs(epsg=4326)

In [ ]:
# plot
fig, ax = plt.subplots(figsize=(10,10))
gdf_counties.plot(ax=ax)
gdf_od.plot(ax=ax, color='red', markersize=0.5)
fig.show()

Now, we can calculate radius of gyration.

In [ ]:
# merge trip and o/d data
trips_od = pd.merge(trips_clean, od, on=["trip_id", "household_id"], how="left")

In [ ]:
def rog(points, centroid):
    '''
    this function is calculates rog given unique points
    points: list of points; centroid: (x,y)
    returns radius of gyration
    '''
    d_i = [geopy.distance.geodesic(points[p], (centroid[1], centroid[0])).miles for p in range(len(points))]
    n = len(d_i)
    sum_di = 0
    for i in range(n):
        sum_di += d_i[i]**2
    rog = math.sqrt(sum_di/n)
     
    return rog

In [ ]:
def rog_centroid_df(row):
    '''
    this function calculates the radius of gyration (ROG) for each person, using UNIQUE points
    person_od_df: df with all trips made by a single individual
    return rog_centroid, rog_home: ROG based around centroid, ROG based around home 
    '''
    # filter df by personid
    person_trips_df = trips_od[trips_od.person_dim_id == row.person_id]

    # get uique points
    unique_lat = set()
    unique_lng = set()
    unique_pts = set()
    for idx, row in person_trips_df.iterrows():
        olat = round(row.origin_lat, 3)
        olng = round(row.origin_lng, 3)
        dlat = round(row.dest_lat, 3)
        dlng = round(row.dest_lng, 3)
        unique_lat.add(olat)
        unique_lat.add(dlat)
        unique_lng.add(olng)
        unique_lng.add(dlng)
        unique_pts.add((olat, olng))
        unique_pts.add((dlat, dlng))
    # calculate centroid lat/lon
    centroid = (np.mean(list(unique_lng)), np.mean(list(unique_lat)))
    
    try:
        # calculate ROG around centroid
        rog_centroid = rog(list(unique_pts), centroid)
        return rog_centroid
    except:
        return None

In [ ]:
persons["rog"] = persons.apply(rog_centroid_df, axis=1)

Trace data is also available for 2017 data (and for 2019, though we do not have that data from PSRC). See more examples of trace data with Minda & Jaime's work from the summer.